# Bayesian Analysis of Clinical Scenario in "Don't Drop The Anchor" by Manesh, R.S., et. al. (2014)

### Introduction

In [1]:
import pymc3 as pm
import arviz as az
import numpy as np

In [2]:
def run_model():
    likelihood_ca19_9 = 0.25
    likelihood_fna = 0.11
    certainty = 1
    alpha = 9 * certainty
    beta = 1 * certainty
    mean_prior = alpha / (alpha + beta)
    var_prior = (alpha * beta) / (np.power(alpha + beta, 2) * (alpha + beta + 1))

    with pm.Model() as model:
        # pre_test prob 1
        p_1 = pm.Beta('p_1', alpha=alpha, beta=beta)
        # p_1 = pm.Uniform('p_1')
        o_1 = pm.Deterministic('o_1', (p_1 / (1 - p_1)))
        l_ca19_9 = pm.HalfNormal('l_ca19_9', sigma=likelihood_ca19_9)

        # prob after CA-19-9
        o_2 = pm.Deterministic('o_2', o_1 * l_ca19_9)
        p_2 = pm.Deterministic('p_2', o_2 / (o_2 + 1))

        # prob after FNA
        l_fna = pm.HalfNormal('l_fna', sigma=likelihood_fna)
        o_3 = pm.Deterministic('o_3', o_2 * l_fna)
        p_3 = pm.Deterministic('p_3', o_3 / (o_3 + 1))

        model_trace = pm.sample(cores=1, chains=1, draws=10000)
    
    return model_trace

In [ ]:
model_trace = run_model()

INFO (theano.gof.compilelock): Waiting for existing lock by process '32479' (I am process '33406')
INFO (theano.gof.compilelock): To manually release the lock, delete /Users/johnwilliam/.theano/compiledir_macOS-10.15.5-x86_64-i386-64bit-i386-3.8.2-64/lock_dir


In [ ]:
az.plot_trace(model_trace,var_names=['p_1','l_ca19_9','p_2'])
az.plot_trace(model_trace,var_names=['p_2','l_fna','p_3'])
az.plot_trace(model_trace,var_names=['p_1','p_2','p_3'])